<font size='6'>BD3P2 - Pyspark ETL Assignment</font>  
<font size='5'>GROUP-10</font>  
<font size='3'><b>Dataset:</b> Titanic - db table</font>  

<b>i.Extract: Load the data</b>  
     - Read data as pandas dataframe   
     - then create spark dataframe   
     - create a table view "titanic" as spark SQL  
<b>ii.Transform: Exploratory data analysis using spark df</b>  
     - Unique passengerId count  
     - GroupBy sex and count of survived  
     - GroupBy Pclass and sum of Fare  
     - Update column age values as 1,2,3 ->child<10,teen>10 to <25,adult>25  
     - GroupBy age,Embarked,Pclass and count of survived  
<b>iii.Load: Save analysis report</b>  
     - save as tables - partitioned by sex

<font size='4'><b>i. Extract: Load the data</b></font>

<font size='4'>a. Read data as pandas dataframe</font>

In [ ]:
from random import random
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
df_path = 'C:/Users/Jerome/Downloads/pyspark_data/titanic.csv'
data = pd.read_csv(df_path)

In [ ]:
print(data)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [ ]:
os.environ['PYSPARK SUBMIT ARGS'] = " --master local[2] pyspark-shell"
os.environ['JAVA_HOME'] = 'C:/Program Files/Java/jdk1.8.0_311'

<font size='4'>b. Creating spark dataframe</font>

In [ ]:
#Creating a spark Session
spark_session = SparkSession.builder.master('local').appName('titanic_csv').getOrCreate()
sc = spark_session.sparkContext

In [ ]:
titanic_data = spark_session.read.csv(df_path ,sep=',',inferSchema=True, header=True ,nullValue='')

In [ ]:
print(titanic_data)

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]


In [ ]:
print(titanic_data.show())

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [ ]:
titanic_data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
titanic_data.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

<font size='4'>c. Create a table view "titanic" as spark SQL</font>

In [ ]:
sqlContext = SQLContext(sc)
sqlContext.registerDataFrameAsTable(titanic_data, "titanic_table")

C:\Users\Jerome\anaconda3\lib\site-packages\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
# Using query show table
sqlContext.sql("SELECT * FROM titanic_table ")

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [ ]:
# Using query show spark netflix data
titanic = sqlContext.sql("SELECT * FROM titanic_table ")
titanic.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

<font size='4'><b>ii.Transform: Exploratory data analysis using spark df</b></font>

<font size='4'>Unique passengerId count</font>

In [ ]:
Unique_Id_count = sqlContext.sql("select distinct count(PassengerId) as Total from titanic_table")
Unique_Id_count.show()

+-----+
|Total|
+-----+
|  891|
+-----+



In [ ]:
#Distinct count through spark dataframe
titanic.distinct().count()

891

<font size='4'>GroupBy sex and count of survived</font>

In [ ]:
#
titanic.filter(titanic.Survived==1).groupBy("Sex").count().show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  233|
|  male|  109|
+------+-----+



In [ ]:
#
titanic.groupBy("Sex", "Survived").count().show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  233|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+



<font size='4'>GroupBy Pclass and sum of Fare</font>

In [ ]:
#
titanic.groupBy("Pclass").sum("Fare").show()

+------+------------------+
|Pclass|         sum(Fare)|
+------+------------------+
|     1|18177.412499999984|
|     3| 6714.695100000002|
|     2|3801.8416999999995|
+------+------------------+



<font size='4'>Update column age values as 1,2,3 ->child<10, teen>10 to <25, adult>25</font>

In [ ]:
titanic.agg({'Age': 'max'}).show()

+--------+
|max(Age)|
+--------+
|    80.0|
+--------+



In [ ]:
titanic = titanic.withColumn("Age", titanic['Age'].cast("int"))
titanic = titanic.withColumn("Age", titanic['Age'].cast("string"))
for i in range(25,81):
    titanic = titanic.withColumn('Age', regexp_replace('Age', str(i), 'Adult'))
for i in range(10,25):
    titanic = titanic.withColumn('Age', regexp_replace('Age', str(i), 'Teen'))
for i in range(0,10):
    titanic = titanic.withColumn('Age', regexp_replace('Age', str(i), 'Child'))
titanic.show()

+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|  Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| Teen|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|Adult|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|Adult|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|Adult|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|Adult|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male| null|    0|   

<font size='4'>GroupBy age,Embarked,Pclass and count of survived</font>

In [ ]:
titanic.filter(titanic.Survived==1).groupBy("Age", "Embarked", "Pclass").count().show()

+-----+--------+------+-----+
|  Age|Embarked|Pclass|count|
+-----+--------+------+-----+
|Adult|       S|     1|   49|
| Teen|       S|     2|   15|
| Teen|       Q|     3|    5|
|Adult|       Q|     2|    1|
|Adult|       Q|     3|    1|
|Child|       C|     2|    2|
| Teen|       S|     3|   18|
|Child|       C|     3|    6|
| Teen|       S|     1|   15|
|Adult|       C|     2|    2|
|Child|       S|     2|   15|
|Child|       S|     3|   13|
|Adult|       C|     3|    3|
| Teen|       C|     1|   14|
|Adult|       Q|     1|    1|
|Child|       S|     1|    2|
|Adult|       S|     2|   44|
| Teen|       C|     2|    4|
|Adult|       C|     1|   39|
| Teen|       C|     3|    9|
+-----+--------+------+-----+
only showing top 20 rows



<font size='4'><b>iii.Load: Save analysis report df</b></font>

In [ ]:
titanic.write.option("header",True).partitionBy("Sex").mode("overwrite").csv('./output')